# OLS

The linear method makes use of the log backward return (log price difference) to predict foward return.

Training:
1) Ridge regression: on 30 features
2) PC regression: pca on 30 features then perform ols

Feature: 10 stocks, each with 3 backward return (say, 3min, 7min, 10min, see correlation to decide)

Response: 10 stocks' 30min forward return. 

Groups: [1,4,5,6,8],[0,2,3,7,9]

## Data Preparation

In [1]:
import os
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

log_pr = pd.read_pickle("../data/log_price.df")
volu = pd.read_pickle("../data/volume_usd.df")

In [2]:
# def wide_format(df):
#     df_= df.reset_index(level=['stock']).sort_index()
#     df_ = df_.pivot(columns ='stock')
#     df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

#     return df_


# def get_feature_train(log_pr, volu, x_begin_idx, x_end_idx, y_begin_idx, 
#                         grp_idx=None, rm_outlier=False, print_cor=True):
#     """
#     Input:
#     log_pr (pdSeries): train set
#     volu (pdSeries): train set
#     x_begin_idx (pdIndex): to truncate the NaNs
#     grp_idx (dict): key is group idx, value is list of stock idx

#     Returns:
#     feature_dict (dict): key is group idx, value is a tuple of feature matrix and response
#     """

#     log_pr_df = log_pr.reset_index().melt(id_vars=['timestamp'])
#     log_pr_df.columns = ['timestamp', 'stock', 'log_pr']
#     log_pr_df = log_pr_df.set_index(['timestamp', 'stock']).sort_index()

#     volu_df = volu.reset_index().melt(id_vars=['timestamp'])
#     volu_df.columns = ['timestamp', 'stock', 'volu']
#     volu_df = volu_df.set_index(['timestamp', 'stock']).sort_index()

#     features = pd.DataFrame(index=log_pr_df.index)
#     # log_pr feature
#     # for i in [10, 20, 30]:
#     #     features['log_pr_{}'.format(i)] = -log_pr_df.groupby(level='stock').log_pr.diff(i)
#     features['log_pr_{}'.format(30)] = -log_pr_df.groupby(level='stock').log_pr.diff(30)

#     std_10 = lambda x: x.rolling(10).std()
#     features['log_pr_std_10'] = log_pr_df.groupby(level='stock').log_pr.apply(std_10)

#     # volume feature
#     log_fn = lambda x: np.log(x+1)
#     features['log_volu'] = volu_df.groupby(level='stock').volu.apply(log_fn)

#     # stdised volume in 2 hours backward rolling windows
#     zscore_fn = lambda x: (x - x.rolling(window=240, min_periods=20).mean()) / x.rolling(window=240, min_periods=20).std()
#     features['volu_z_score'] = volu_df.groupby(level='stock').volu.apply(zscore_fn)

#     # feature_dropped = features.iloc[x_begin_idx:x_end_idx]
#     response = log_pr.diff(30)

#     if grp_idx is not None:
#         feature_dict = {}
#         for key, idx_lis in grp_idx.items():
#             feature_df_dropped = wide_format(features.loc[pd.IndexSlice[:,idx_lis],:])
#             # transform back to wide format
#             feature_dict[key] = (feature_df_dropped.iloc[x_begin_idx:x_end_idx], 
#                                             response[idx_lis].iloc[y_begin_idx:])
#         return feature_dict
#     else:
#         # transform back to wide format
#         feature_df_dropped = wide_format(features).iloc[x_begin_idx:x_end_idx]
#         # feature_df_dropped = feature_df[x_begin_idx:x_end_idx]
    
#         if print_cor:
#             for i in range(10):
#                 feature_train_0 = features.xs(i, level='stock')
#                 print(feature_train_0.corrwith(response[i]))

#         return feature_df_dropped, response.iloc[y_begin_idx:]

## Feature Engineering

In [53]:
def wide_format(df):
    df_= df.reset_index(level=['stock']).sort_index()
    df_ = df_.pivot(columns ='stock')
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_

def rsi(close_delta, periods=20, ema=True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = close_delta.diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
	    # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

def get_feature_train(log_pr, volu, x_begin_idx, x_end_idx, y_begin_idx, 
                        grp_idx=None, rm_outlier=False, print_cor=True):
    """
    Input:
    log_pr (pdSeries): train set
    volu (pdSeries): train set
    x_begin_idx (pdIndex): to truncate the NaNs
    grp_idx (dict): key is group idx, value is list of stock idx

    Returns:
    feature_dict (dict): key is group idx, value is a tuple of feature matrix and response
    """

    log_pr_df = log_pr.reset_index().melt(id_vars=['timestamp'])
    log_pr_df.columns = ['timestamp', 'stock', 'log_pr']
    log_pr_df = log_pr_df.set_index(['timestamp', 'stock']).sort_index()

    volu_df = volu.reset_index().melt(id_vars=['timestamp'])
    volu_df.columns = ['timestamp', 'stock', 'volu']
    volu_df = volu_df.set_index(['timestamp', 'stock']).sort_index()

    features = pd.DataFrame(index=log_pr_df.index)

    # log_pr feature
    for i in [20, 30]:
        features['log_pr_{}'.format(i)] = -log_pr_df.groupby(level='stock').log_pr.diff(i)

    # # EMA
    # ema = lambda x: x.ewm(span=i).mean()
    # for i in [10, 30, 50]:
    #     features['pr_ema_{}'.format(i)] = log_pr_df.groupby(level='stock').log_pr.apply(ema)

    # # MA
    # for i in [10, 30, 50]:
    #     ma = lambda x: x.rolling(i).mean()
    #     features['pr_ma_{}'.format(i)] = log_pr_df.groupby(level='stock').apply(ma)

    k_period = 40
    d_period = 3
    ma_max = lambda x: x.rolling(k_period).max()
    ma_min = lambda x: x.rolling(k_period).min()
    mad = lambda x: x.rolling(d_period).mean()
    msd = lambda x: x.rolling(d_period).sum()

    features['pr_min_40'] = log_pr_df.groupby(level='stock').log_pr.apply(ma_min)
    features['pr_max_40'] = log_pr_df.groupby(level='stock').log_pr.apply(ma_max)

    features['pr_so_40'] = (log_pr_df.log_pr - features['pr_min_40'])*100 / (features['pr_max_40'] - features['pr_min_40'])
    features['pr_so_40d3'] = features.groupby(level='stock').pr_so_40.apply(mad)

    # STD of log price
    for i in [30]:
        std = lambda x: x.rolling(i).std()
        features['log_pr_std_{}'.format(i)] = log_pr_df.groupby(level='stock').log_pr.apply(std)

    # RSI
    # features['rsi_20'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi)
    features['rsi_30'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=30)
    # features['rsi_50'] = log_pr_df.groupby(level='stock').log_pr.apply(rsi, periods=50)

    # volume feature
    log_fn = lambda x: np.log(x+1)
    features['log_volu'] = volu_df.groupby(level='stock').volu.apply(log_fn)

    # stdised volume in 2 hours backward rolling windows
    zscore_fn = lambda x: (x - x.rolling(window=30, min_periods=20).mean()) / x.rolling(window=240, min_periods=20).std()
    features['volu_z_score'] = volu_df.groupby(level='stock').volu.apply(zscore_fn)

    # # Chaikin's money flow
    # features['mf_40'] = volu_df.volu * ((2*log_pr_df.log_pr - features['pr_min_40'])
    #                             / (features['pr_max_40'] - features['pr_min_40']))
    # features['mf_40_ma'] = (features.groupby(level='stock').mf_40.apply(msd) / 
    #                         volu_df.groupby(level='stock').volu.apply(msd))

    # feature_dropped = features.iloc[x_begin_idx:x_end_idx]
    response = log_pr.diff(30)
    # print(features.shape)
    # print(feature_dropped.shape)
    # print(response_dropped.shape)

    if grp_idx is not None:
        feature_dict = {}
        for key, idx_lis in grp_idx.items():
            feature_df_dropped = wide_format(features.loc[pd.IndexSlice[:,idx_lis],:])
            # transform back to wide format
            feature_dict[key] = (feature_df_dropped.iloc[x_begin_idx:x_end_idx], 
                                            response[idx_lis].iloc[y_begin_idx:])
        return feature_dict
    else:
        # transform back to wide format
        feature_df_dropped = wide_format(features).iloc[x_begin_idx:x_end_idx]
        # feature_df_dropped = feature_df[x_begin_idx:x_end_idx]
    
        if print_cor:
            for i in range(10):
                
                feature_train_0 = features.xs(i, level='stock').iloc[x_begin_idx:x_end_idx]
                print(feature_train_0.corrwith(response[i]))
                print(feature_train_0.isnull().sum())

        return feature_df_dropped, response.iloc[y_begin_idx:]

In [54]:
grp_idx = {i: [i] for i in range(10)}

x_begin_idx = 30
x_end_idx = -30
y_begin_idx = 60

train_split_t = log_pr.index[-87841]
#vali_split_t = log_pr.index[-44641]

train_feature_dict = get_feature_train(log_pr[:train_split_t], volu[:train_split_t], x_begin_idx, x_end_idx, y_begin_idx,
                                        grp_idx=grp_idx, print_cor=False)

vali_feature_dict = get_feature_train(log_pr[train_split_t:], volu[train_split_t:], x_begin_idx, x_end_idx, y_begin_idx,
                                        grp_idx=grp_idx,print_cor=False)

## Modelling

In [4]:
from sklearn.metrics import mean_squared_error
from statsmodels.regression.linear_model import OLS

### Feature Selection with AIC

In [55]:
xtrain0, ytrain0 = train_feature_dict[0]
xvali0, yvali0 = vali_feature_dict[0]

In [57]:
reg0 = OLS(ytrain0.values, xtrain0.values).fit()
# reg0.summary()
# mean_squared_error(ytrain0, reg0.predict())
# mean_squared_error(yvali0, reg0.predict(xvali0))
# np.corrcoef(yvali0.squeeze(), reg0.predict(xvali0).squeeze())

In [64]:
pd.concat((ytrain0,yvali0)).shape

(264841, 1)

In [67]:
import step_reg
step_reg.forward_regression(xtrain0,ytrain0,0.05)

c:\Users\royul\Documents\GitHub\multivariate-time-series-prediction\models(ipynb)\step_reg.py:12: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


ValueError: The indices for endog and exog are not aligned

In [58]:
reg0.aic

-1330510.5344174337

### Train function for the ten stocks

In [ ]:
def train_OLS(feature_dict):
    mod_dict = {}
    for i, (X, y) in feature_dict.items():
        mod_dict[i] = OLS(y.values, X.values).fit()

    return mod_dict

In [ ]:
mod_dict = train_OLS(train_feature_dict)

#### Save Model

In [ ]:
import pickle
with open("models.pckl", "wb") as f:
    for model in mod_dict.values():
        pickle.dump(model, f)

### Evaluation

In [ ]:
def wide_format_test(df):
    df_= df.reset_index()
    df_ = df_.pivot(columns ='index').apply(lambda s: s.dropna().reset_index(drop=True))
    df_.columns = df_.columns.get_level_values(0) + '_' +  [str(x) for x in df_.columns.get_level_values(1)]

    return df_

def get_feature_test(log_pr, volu, grp_idx=None):
    """
    Input: 
    log_pr (pdSeries): 1 day of log pr 
    volu (pdSeries): 1 day of volume

    Output:
    test data frame
    """
    features = pd.DataFrame(index=log_pr.columns)

    # backward return
    # print(-(log_pr.iloc[-1] - log_pr.iloc[-30]).values)
    #for i in [10, 20, 30]:
    #    features['log_pr_{}'.format(i)] = -(log_pr.iloc[-1] - log_pr.iloc[-i]).values
    features['log_pr_{}'.format(30)] = -(log_pr.iloc[-1] - log_pr.iloc[-30]).values
    # backward rolling std
    features['log_pr_std_10'] = log_pr.iloc[-10:].std(0).values
    
    # volume features
    features['log_volu'] = np.log(volu.iloc[-1].values + 1)
    features['volu_z_score'] = ((volu.iloc[-1] - volu.iloc[-240:].mean())/volu.iloc[-240:].std()).values

    if grp_idx is None:
        return wide_format_test(features)
    else:
        df_dict = {}
        for key, idx_lis in grp_idx.items():
            df_dict[key] = wide_format_test(features.loc[idx_lis])
        return df_dict


In [ ]:
model_dict = mod_dict #{i: pickle.load(open('../model/ridge{}.sav'.format(i), 'rb')) for i in range(2)}

def get_r_hat(A, B): 
    """
        A: 1440-by-10 dataframe of log prices with columns log_pr_0, ... , log_pr_9
        B: 1440-by-10 dataframe of trading volumes with columns volu_0, ... , volu_9    
        return: a numpy array of length 10, corresponding to the predictions for the forward 30-minutes returns of assets 0, 1, 2, ..., 9
    """
    grp_idx = {i: [i] for i in range(10)}
    x = get_feature_test(A, B, grp_idx=grp_idx)
    pred_dict = {i: model.predict(x[i]) for i, model in model_dict.items()}
    
    out = np.zeros(10)
    for keys, idx in grp_idx.items():
        out[idx] = pred_dict.get(keys)
    return out

In [ ]:
# def get_r_hat_tune(A, B):
#     # grp_idx = {0:[1,5,6,8], 1:[0,2,3,4,7,9]}
#     x = get_feature_test(A, B)
#     return rr.predict(x)

In [ ]:
def evaluate_tune(log_pr_test, volu_test):

    t0 = time.time()
    dt = datetime.timedelta(days=1)

    r_fwd = (log_pr_test.shift(-30) - log_pr_test).iloc[1440::10]
    # r_fwd = return_true.iloc[1440::10]
    # r_fwd.index = log_pr_test.index[1440::10]
    r_hat = pd.DataFrame(index=log_pr_test.index[1440::10], columns=log_pr_test.columns, dtype=np.float64)

    for t in log_pr_test.index[1440::10]: # compute the predictions every 10 minutes
        # inputs 1 day of log price and volume
        r_hat.loc[t, :] = get_r_hat(log_pr_test.loc[(t - dt):t], volu_test.loc[(t - dt):t])
    t_used = time.time() - t0
    print("Time used: ", t_used)

    r_fwd_all = r_fwd.iloc[:-3].values.ravel() # the final 3 rows are NaNs. 
    r_hat_all = r_hat.iloc[:-3].values.ravel()
    return r_fwd.corrwith(r_hat), np.corrcoef(r_fwd_all, r_hat_all)[0,1]

In [ ]:
log_pr_test = log_pr
volu_test = volu

In [ ]:
evaluate_tune(log_pr_test, volu_test)

In [ ]:
log_pr_train_vali = log_pr[:train_split_t]
volu_train_vali = volu[:train_split_t]

In [ ]:
evaluate_tune(log_pr_train_vali, volu_train_vali)